# Information

**Author:**<br>Pascal Munaretto (<a href="mailto:pascal.munaretto@outlook.com">Mail</a>)

**Date:**<br>30.09.2022

**Type:**<br>Master's Thesis

**Topic:**<br>Design, Implementation and Performance Analysis of an AI-Based Insider Threat Detection Platform	in Splunk To Counteract Data Exfiltration

**Study Program:**<br>Enterprise and IT Security

**Institution:**<br><a href="https://www.hs-offenburg.de">Offenburg University of Applied Sciences</a>

**Github:**<br>https://github.com/pmunaretto/Master-Thesis

# Setup

## Requirements

In [ ]:
!pip install pyod suod

## Patches

In [ ]:
# Add callbacks to Auto Encoder, VAE and Deep SVDD
!cp /content/drive/MyDrive/CERT/patches/patched_auto_encoder.py /usr/local/lib/python3.7/dist-packages/pyod/models/auto_encoder.py
!cp /content/drive/MyDrive/CERT/patches/patched_vae.py /usr/local/lib/python3.7/dist-packages/pyod/models/vae.py
!cp /content/drive/MyDrive/CERT/patches/patched_deep_svdd.py /usr/local/lib/python3.7/dist-packages/pyod/models/deep_svdd.py

## Imports

In [ ]:
import os
import math
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from timeit import default_timer as timer
from random import seed, randint
from sklearn.base import TransformerMixin, BaseEstimator, clone
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import ParameterGrid
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, recall_score, f1_score, precision_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.compose import make_column_selector as selector
from matplotlib.backends.backend_pgf import FigureCanvasPgf
from matplotlib.ticker import PercentFormatter
from pyod.models.iforest import IForest
from pyod.models.ecod import ECOD
from pyod.models.copod import COPOD
from pyod.models.loda import LODA
from pyod.models.cblof import CBLOF
from pyod.models.pca import PCA
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.vae import VAE
from pyod.models.deep_svdd import DeepSVDD
from IPython.display import display, Markdown

## Configuration

In [ ]:
matplotlib.backend_bases.register_backend("pgf", FigureCanvasPgf)

plt.rcParams.update({
    "figure.dpi": 100,
    "savefig.dpi": 300,
    "font.size": 12,
    "image.cmap": "plasma",
    "axes.prop_cycle": plt.cycler("color", "bgrcmyk"), 
    "pgf.texsystem": "pdflatex",
    "font.family": "serif",
    "text.usetex": True,
    "pgf.rcfonts": False
})

tf.get_logger().setLevel("WARN")

# Global Configuration
BASE_PATH     = "/content/drive/MyDrive/CERT/r4.2"
N_JOBS        = -1
N_ITER        = 5
CONTAMINATION = 0.01
RETRAIN       = False
DATASET_NAME  = "logon_role"
FEATURE_SETS  = [
    ["activity", "hour"],
    ["activity", "weekday"],
    ["activity", "hour", "weekday"],
    ["activity", "hour_sin", "hour_cos"],
    ["activity", "weekday_sin", "weekday_cos"],
    ["activity", "hour_sin", "hour_cos", "weekday_sin", "weekday_cos"]
]

## Helper Functions

In [ ]:
class Debugger(BaseEstimator, TransformerMixin):

    def transform(self, data):
        print("Shape of Preprocessed data:", data.shape)
        print(pd.DataFrame(data).head())
        return data

    def fit(self, data, y=None, **fit_params):
        return self


def plot_anomaly_scores(series, identifier, min, max, save=True):
    plt.figure(figsize=(10,3))
    plt.hist(
        series,
        weights=np.ones(len(series)) / len(series),
        bins=np.arange(min, max, 0.02),
        rwidth=0.8
    )
    plt.xlim(xmin=min, xmax=max)
    plt.xticks(np.arange(min, max+0.1, 0.1))
    plt.xlabel("Anomaly Score")
    plt.ylabel("Percentage")
    plt.gca().yaxis.set_major_formatter(PercentFormatter(1))
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def plot_confusion_matrix(y_true, y_pred, identifier, save=True):
    ConfusionMatrixDisplay.from_predictions(
        y_true,
        y_pred,
        labels=[0, 1],
        display_labels=["Benign", "Malicious"],
        values_format="d",
        colorbar=True,
        cmap="plasma_r"
    )
    plt.grid(False)
    if save:
        plt.savefig(os.path.join(BASE_PATH, "figures", f"{identifier}.pgf"), format="pgf")
    plt.show()


def print_training_result(metrics):
    print(
        "  ".join(
            [
                f"\033[1;33m Training Time: {metrics.training_time_avg:<7.4f}\033[0m",
                f"\033[1;33m Inference Time: {metrics.inference_time_avg:<7.4f}\033[0m",
                f"\033[1;35m pAUC: {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}\033[0m",
                f"\033[1;35m Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}\033[0m",
                f"\033[1;32m TN: {metrics.best_classifier_TN:<6}\033[0m",
                f"\033[1;31m FP: {metrics.best_classifier_FP:<5}\033[0m",
                f"\033[1;31m FN: {metrics.best_classifier_FN:<3}\033[0m",
                f"\033[1;32m TP: {metrics.best_classifier_TP:<3}\033[0m",
                f"\033[1;37m Params: {metrics.name}\033[0m"
            ]
        )
    )


def print_gridsearch_result(metrics):
    print(
        "\n".join(
            [
                "\n\033[4mBest hyperparameters:\033[0m",
                f"Params: {metrics.name}",
                f"pAUC:   {metrics.p_auc_10_avg:02.4f} \u00B1 {metrics.p_auc_10_std:02.4f}",
                f"Recall: {metrics.recall_avg:02.4f} \u00B1 {metrics.recall_std:02.4f}"
            ]
        )
    )


def calculate_dispersion_metrics_for_columns(source_df, destination_df, columns):
    for column in columns:
        avg = np.average(source_df[column])
        std = np.std(source_df[column])
        destination_df[f"{column}_avg"] = avg
        destination_df[f"{column}_std"] = std if not math.isnan(std) else 0

    return destination_df


def train_classifier_on_single_users(df, classifier, features, params, n_iter=10):
    # Create a dataframe where the results of the different seeds will be stored
    random_state_summary = pd.DataFrame()

    # Reset the PRNG seed
    seed(1)

    # Group the dataframe by users
    grouped = df.groupby("functional_unit", as_index=False)

    # Perform the training process multiple times with random seeds
    for _ in range(n_iter):

        # Create a dataframe where the results of the classifiers will be stored
        user_timings = pd.DataFrame()
        user_predictions = pd.DataFrame()

        for name, group in grouped:

            # Create a clone of the classifier
            try:
                classifier = clone(classifier)
            except:
                pass
            
            # Update the parameters of the classifier according to the grid search
            classifier.set_params(**params)
            
            # Set the random state attribute of the classifier (if it has one)
            try:
                classifier.set_params(**{"random_state": randint(0, 2**32)})
            except Exception:
                pass

            # Define the transformers that do the rest of the preprocessing (scaling, encoding)
            numeric_transformer = Pipeline(steps=[
                ("scaler", RobustScaler())
            ])
            categorical_transformer = Pipeline(steps=[
                ("ohe", OneHotEncoder())
            ])

            # Filter the dataframe according to the features 
            group_filtered = group[features].copy()

            # Create a pipeline that performs the feature selection and scaling
            pipe = Pipeline([
                ("column_transformer", ColumnTransformer(
                    transformers=[
                        ("num", numeric_transformer, selector(dtype_exclude=["category", "object"])),
                        ("cat", categorical_transformer, selector(dtype_include=["category", "object"]))
                    ]
                )),
                ("classifier", classifier)
            ])
        
            # Benchmark the training
            start_training = timer()
            pipe.fit(group_filtered)
            end_training = timer()

            # Benchmark the inference
            start_inference = timer()
            pipe.predict(group_filtered)
            end_inference = timer()

            # Add the predictions and anomaly scores to the group dataframe
            group["y_true"] = group["threat"]
            group["y_pred"] = pipe.named_steps["classifier"].labels_
            group["scores"] = pipe.named_steps["classifier"].decision_scores_

            # Create a new series with the training metrics for the user iteration
            timings = pd.Series(
                {
                    "training_time": end_training - start_training,
                    "inference_time": end_inference - start_inference
                }
            )

            # Append the series to our user summary dataframes
            user_timings = user_timings.append(timings, ignore_index=True)
            user_predictions = pd.concat([user_predictions, group])

        # If there are inf values in the anomaly scores, replace it with the elsewise highest value
        max_without_infs = user_predictions["scores"].replace([np.inf, -np.inf], np.nan).max()
        user_predictions["scores"].replace(np.inf, max_without_infs, inplace=True)

        # Find the right threshold to satisfy the contamination and add the final predictions
        upper = max_without_infs
        lower = 0
        for i in range(0,500):
            threshold = (upper + lower) / 2
            test = (user_predictions.scores > threshold).astype(bool)
            if test.sum() > CONTAMINATION * len(df):
                lower = threshold
            else:
                upper = threshold
        user_predictions["y_pred"] = test

        # After the predictions were added, calculate the evaluation metrics
        recall = recall_score(user_predictions.threat, user_predictions.y_pred)
        precision = precision_score(user_predictions.threat, user_predictions.y_pred)
        f1 = f1_score(user_predictions.threat, user_predictions.y_pred)
        cm = confusion_matrix(user_predictions.threat, user_predictions.y_pred, labels=[0, 1])

        # Try to calculate the AUC, this could fail if one of the anomaly scores is infinite
        try:
            auc = roc_auc_score(user_predictions.threat, user_predictions.scores)
            p_auc_10 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.1)
            p_auc_20 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.2)
            p_auc_30 = roc_auc_score(user_predictions.threat, user_predictions.scores, max_fpr=0.3)
        except ValueError as e:
            print(e)

        # Create a new series with all the information about the iteration
        metrics = pd.Series(
            {
                "training_time": user_timings.training_time.sum(),
                "inference_time": user_timings.inference_time.sum(),
                "recall": recall,
                "precision": precision,
                "f1": f1,
                "TN": cm[0][0],
                "FP": cm[0][1],
                "FN": cm[1][0],
                "TP": cm[1][1],
                "auc": auc,
                "p_auc_10": p_auc_10,
                "p_auc_20": p_auc_20,
                "p_auc_30": p_auc_30,
                "y_true": user_predictions.y_true,
                "y_pred": user_predictions.y_pred,
                "scores": user_predictions.scores,
            }
        )

        # Append the series to our summary dataframe
        random_state_summary = random_state_summary.append(metrics, ignore_index=True)

    # Convert the confusion matrix to integers
    random_state_summary = random_state_summary.astype({"TN": "int32", "FP": "int32", "FN": "int32", "TP": "int32"})

    # Locate the best classifier and separate the predictions from it
    results = random_state_summary.iloc[random_state_summary.p_auc_10.argmax()].rename(str(params))
    predictions = results.loc[["y_true", "y_pred", "scores"]]

    # Remove the columns that should not be part of the results dataframe
    results.drop(["y_true", "y_pred", "scores"], inplace=True)

    # Add the prefix
    results = results.add_prefix("best_classifier_")
    
    # Add the average training and inference time to the dataframe
    results["training_time_avg"]  = np.average(random_state_summary["training_time"])
    results["inference_time_avg"] = np.average(random_state_summary["inference_time"])

    # Calculate averages and different dispersion metrics for the best classifier series
    results = calculate_dispersion_metrics_for_columns(
        source_df=random_state_summary,
        destination_df=results,
        columns=["auc", "p_auc_10", "p_auc_20", "p_auc_30", "recall"]
    )

    return results, predictions


class GridSearch:
    def __init__(self, df, classifier, features, parameters, gridsearch_path):

        # Instance variables
        self.df = df
        self.classifier = classifier
        self.features = features
        self.parameters = parameters
        self.gridsearch_path = gridsearch_path

        # Main paths
        self.summary_path = os.path.join(self.gridsearch_path, "gridsearch_summary.csv")
        self.best_results_path = os.path.join(self.gridsearch_path, "best_results.csv")
        self.best_preds_path = os.path.join(self.gridsearch_path, "best_preds.csv")

        # Create the output directory for the gridsearch
        os.makedirs(gridsearch_path, exist_ok=True)

        # Read existing files
        if os.path.exists(self.summary_path) and not RETRAIN:
            self.gridsearch_summary = pd.read_csv(self.summary_path, index_col=0)
        else:
            self.gridsearch_summary = pd.DataFrame()
        if os.path.exists(self.best_results_path) and not RETRAIN:
            self.best_results = pd.read_csv(self.best_results_path, squeeze=True, index_col=0)
        else:
            self.best_results = None


    def start_training(self):

        # Create an iterable parameter grid from the parameters dictionary
        grid = ParameterGrid(self.parameters)

        # Debug output
        print(f"\033[4mTesting {len(list(grid))} different hyperparameter combinations\033[0m")

        # Iterate over all possible parameter combinations
        for params in grid:

            # Skip the parameters if they are already part of the gridsearch summary
            if not RETRAIN and not self.gridsearch_summary.empty and str(params) in self.gridsearch_summary.index:
                print_training_result(self.gridsearch_summary.loc[str(params)])
                continue

            # Start the training process
            try: 
                if hasattr(classifier, "random_state"):
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        features=self.features,
                        params=params,
                        n_iter=N_ITER
                    )
                else:
                    results, predictions = train_classifier_on_single_users(
                        df=self.df,
                        classifier=self.classifier,
                        features=self.features,
                        params=params,
                        n_iter=1
                    )
            except ValueError as e:
                print(f"Skipping {params}: {e}")
                continue

            # Print the metrics of the best classifier
            print_training_result(results)

            # Add the results to the gridsearch summary
            self.gridsearch_summary = self.gridsearch_summary.append(results)

            # Update the best classifier if the iterations performs better than the current best 
            if self.best_results is None or results.p_auc_10_avg > self.best_results.p_auc_10_avg:
                self.best_results = results
                self.best_preds = predictions
                self.save_best_results()

            # Save the progress
            self.save_gridsearch_summary()

        # Print the results of the gridsearch (parameters with the best average)
        print_gridsearch_result(self.best_results)

        return self.best_results


    def save_gridsearch_summary(self):
        self.gridsearch_summary.to_csv(self.summary_path)


    def save_best_results(self):
        self.best_results.to_csv(self.best_results_path)
        self.best_preds.to_frame()\
            .transpose()\
            .apply(pd.Series.explode)\
            .reset_index(drop=True)\
            .to_csv(self.best_preds_path, index=False)


    def get_summary(self):
        return self.gridsearch_summary


def initiate_training_run(classifier_name, classifier, parameters):
    # Define output paths
    summary_path = os.path.join(BASE_PATH, "results_summary", DATASET_NAME)
    summary_file = os.path.join(summary_path, "summary.csv")

    # Create the output directory for the classifier
    os.makedirs(summary_path, exist_ok=True)

    # Iterate through the feature sets
    for i, features in enumerate(FEATURE_SETS, start=1):

        display(Markdown(f"# {i}/{len(FEATURE_SETS)} - Features: {', '.join(features)}"))

        # Perform a grid search to find the best parameters for the classifier
        gridsearch = GridSearch(
            df=df,
            classifier=classifier,
            features=features,
            parameters=parameters,
            gridsearch_path=os.path.join(BASE_PATH, "results_summary", DATASET_NAME, classifier_name, f"gridsearch{i}")
        )
    
        best_parameter_series = gridsearch.start_training()

        # Read the summary file if it already exists, otherwise create a new one
        if os.path.exists(summary_file):
            summary = pd.read_csv(summary_file, index_col=0)
        else:
            summary = pd.DataFrame()

        # Set the index of the pandas series and update / append it to the summary
        index_name = f"{classifier_name}_dataset{i}"
        best_parameter_series.rename(index_name, inplace=True)
        if index_name in summary.index:
            summary.loc[index_name] = best_parameter_series
        else:
            summary = summary.append(best_parameter_series)

        # Save the summaries and predictions to a file
        summary.sort_index(inplace=True)
        summary.to_csv(summary_file)

## Loading the Data

In [ ]:
import pandas as pd
import os
BASE_PATH     = "/content/drive/MyDrive/CERT/r4.2"


In [ ]:
df.role.nunique()

42

In [ ]:
# Read the dataset
df = pd.read_parquet(os.path.join(BASE_PATH, "preprocessed", "logon"))

# Training - Isolation Forest

In [ ]:
# Local configuration
classifier_name = "isolation_forest"

# Define the classifier that will be used for training
classifier = IForest(
    behaviour="new",
    max_features=1.0,
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_estimators": [1, 10, 50, 100],
    "max_samples": [128, 256, 512, 1024, 2048, 4096],
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 24 different hyperparameter combinations
 Training Time: 0.8760    Inference Time: 0.4833    pAUC: 0.6353 ± 0.0299   Recall: 0.1545 ± 0.0806   TN: 854227   FP: 434     FN: 190   TP: 8     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.7290    Inference Time: 1.9990    pAUC: 0.6888 ± 0.0123   Recall: 0.2182 ± 0.0775   TN: 848212   FP: 6449    FN: 163   TP: 35    Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 19.6508   Inference Time: 8.7730    pAUC: 0.6981 ± 0.0129   Recall: 0.3242 ± 0.0288   TN: 848574   FP: 6087    FN: 127   TP: 71    Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 37.4169   Inference Time: 17.1347   pAUC: 0.6963 ± 0.0058   Recall: 0.3455 ± 0.0122   TN: 848337   FP: 6324    FN: 128   TP: 70    Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.8944    Inference Time: 0.4845    pAUC: 0.6951 ± 0.0331   Recall: 0.1818 ± 0.0925   TN: 853837   FP: 824     FN: 190   TP: 8     Params: {'max_sample

# 2/6 - Features: activity, weekday

Testing 24 different hyperparameter combinations
 Training Time: 0.8002    Inference Time: 0.4456    pAUC: 0.5133 ± 0.0269   Recall: 0.0061 ± 0.0049   TN: 851816   FP: 2845    FN: 196   TP: 2     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.0036    Inference Time: 1.6987    pAUC: 0.5198 ± 0.0213   Recall: 0.0010 ± 0.0020   TN: 849036   FP: 5625    FN: 198   TP: 0     Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 16.2879   Inference Time: 7.2445    pAUC: 0.5206 ± 0.0114   Recall: 0.0000 ± 0.0000   TN: 848362   FP: 6299    FN: 198   TP: 0     Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 31.3210   Inference Time: 14.1441   pAUC: 0.5307 ± 0.0040   Recall: 0.0000 ± 0.0000   TN: 848684   FP: 5977    FN: 198   TP: 0     Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.7983    Inference Time: 0.4501    pAUC: 0.5103 ± 0.0098   Recall: 0.0101 ± 0.0106   TN: 854011   FP: 650     FN: 198   TP: 0     Params: {'max_sample

# 3/6 - Features: activity, hour, weekday

...

Testing 24 different hyperparameter combinations
 Training Time: 0.9006    Inference Time: 0.4844    pAUC: 0.6322 ± 0.0407   Recall: 0.0869 ± 0.0731   TN: 853576   FP: 1085    FN: 172   TP: 26    Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.6171    Inference Time: 2.0087    pAUC: 0.6910 ± 0.0254   Recall: 0.1586 ± 0.0240   TN: 846905   FP: 7756    FN: 166   TP: 32    Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 19.5774   Inference Time: 8.7836    pAUC: 0.7277 ± 0.0249   Recall: 0.2030 ± 0.0258   TN: 847043   FP: 7618    FN: 155   TP: 43    Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 37.8330   Inference Time: 17.2034   pAUC: 0.7318 ± 0.0165   Recall: 0.2030 ± 0.0195   TN: 847330   FP: 7331    FN: 158   TP: 40    Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.9012    Inference Time: 0.4930    pAUC: 0.6853 ± 0.0419   Recall: 0.0687 ± 0.0645   TN: 851898   FP: 2763    FN: 191   TP: 7     Params: {'max_sample

# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 24 different hyperparameter combinations
 Training Time: 0.8545    Inference Time: 0.4640    pAUC: 0.5097 ± 0.0098   Recall: 0.0182 ± 0.0040   TN: 850689   FP: 3972    FN: 194   TP: 4     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 4.0589    Inference Time: 1.7310    pAUC: 0.5434 ± 0.0194   Recall: 0.0273 ± 0.0040   TN: 847852   FP: 6809    FN: 193   TP: 5     Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 16.6267   Inference Time: 7.3969    pAUC: 0.5577 ± 0.0144   Recall: 0.0091 ± 0.0059   TN: 850176   FP: 4485    FN: 197   TP: 1     Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 32.2543   Inference Time: 14.4464   pAUC: 0.5458 ± 0.0124   Recall: 0.0051 ± 0.0064   TN: 849648   FP: 5013    FN: 198   TP: 0     Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 0.8464    Inference Time: 0.4632    pAUC: 0.5306 ± 0.0145   Recall: 0.0152 ± 0.0085   TN: 853210   FP: 1451    FN: 198   TP: 0     Params: {'max_sample

# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 24 different hyperparameter combinations
 Training Time: 1.0405    Inference Time: 0.5308    pAUC: 0.5870 ± 0.0180   Recall: 0.0091 ± 0.0137   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'max_samples': 128, 'n_estimators': 1}
 Training Time: 5.4100    Inference Time: 2.2769    pAUC: 0.6574 ± 0.0139   Recall: 0.1343 ± 0.0204   TN: 847456   FP: 7205    FN: 166   TP: 32    Params: {'max_samples': 128, 'n_estimators': 10}
 Training Time: 22.7956   Inference Time: 10.1576   pAUC: 0.7113 ± 0.0111   Recall: 0.2101 ± 0.0238   TN: 846297   FP: 8364    FN: 157   TP: 41    Params: {'max_samples': 128, 'n_estimators': 50}
 Training Time: 44.0992   Inference Time: 19.7357   pAUC: 0.7194 ± 0.0151   Recall: 0.2232 ± 0.0173   TN: 846283   FP: 8378    FN: 149   TP: 49    Params: {'max_samples': 128, 'n_estimators': 100}
 Training Time: 1.0531    Inference Time: 0.5377    pAUC: 0.6038 ± 0.0200   Recall: 0.0606 ± 0.0437   TN: 851286   FP: 3375    FN: 176   TP: 22    Params: {'max_sample

# Training - LODA

In [ ]:
# Local configuration
classifier_name = "loda"

# Define the classifier that will be used for training
classifier = LODA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_bins": [6, 8, 10, 12, 14, 16, 20],
    "n_random_cuts": [25, 50, 75, 100]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 28 different hyperparameter combinations
 Training Time: 2.6412    Inference Time: 1.6036    pAUC: 0.6067 ± 0.0000   Recall: 0.1717 ± 0.0000   TN: 823978   FP: 30683   FN: 164   TP: 34    Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 4.8397    Inference Time: 2.8528    pAUC: 0.5755 ± 0.0000   Recall: 0.1515 ± 0.0000   TN: 848343   FP: 6318    FN: 168   TP: 30    Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 6.9578    Inference Time: 4.0688    pAUC: 0.5917 ± 0.0000   Recall: 0.0556 ± 0.0000   TN: 847914   FP: 6747    FN: 187   TP: 11    Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 9.0777    Inference Time: 5.3065    pAUC: 0.6681 ± 0.0000   Recall: 0.2020 ± 0.0000   TN: 846996   FP: 7665    FN: 158   TP: 40    Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 2.5819    Inference Time: 1.6608    pAUC: 0.5839 ± 0.0000   Recall: 0.1919 ± 0.0000   TN: 852570   FP: 2091    FN: 160   TP: 38    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 2/6 - Features: activity, weekday

Testing 28 different hyperparameter combinations
 Training Time: 2.6594    Inference Time: 1.5913    pAUC: 0.5039 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 839867   FP: 14794   FN: 195   TP: 3     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 5.0726    Inference Time: 2.8399    pAUC: 0.5382 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 7.0408    Inference Time: 4.1186    pAUC: 0.4986 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 830846   FP: 23815   FN: 192   TP: 6     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 9.1606    Inference Time: 5.3016    pAUC: 0.5191 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 839193   FP: 15468   FN: 195   TP: 3     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 2.5885    Inference Time: 1.6247    pAUC: 0.5013 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 3/6 - Features: activity, hour, weekday

...

Testing 28 different hyperparameter combinations
 Training Time: 2.4685    Inference Time: 1.7161    pAUC: 0.5013 ± 0.0000   Recall: 0.0051 ± 0.0000   TN: 850773   FP: 3888    FN: 197   TP: 1     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 4.5776    Inference Time: 2.9420    pAUC: 0.4973 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 840060   FP: 14601   FN: 194   TP: 4     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 6.5259    Inference Time: 4.2003    pAUC: 0.5464 ± 0.0000   Recall: 0.0354 ± 0.0000   TN: 846567   FP: 8094    FN: 191   TP: 7     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 8.7038    Inference Time: 5.5061    pAUC: 0.5700 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 845526   FP: 9135    FN: 196   TP: 2     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 2.5699    Inference Time: 1.7051    pAUC: 0.6621 ± 0.0000   Recall: 0.1364 ± 0.0000   TN: 844160   FP: 10501   FN: 171   TP: 27    Params: {'n_bins': 8, 'n_random_cuts': 25}


# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 28 different hyperparameter combinations
 Training Time: 2.4538    Inference Time: 1.6558    pAUC: 0.4850 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 4.5548    Inference Time: 2.8613    pAUC: 0.4949 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 854661   FP: 0       FN: 198   TP: 0     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 6.5212    Inference Time: 4.1559    pAUC: 0.4834 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 824363   FP: 30298   FN: 194   TP: 4     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 8.5792    Inference Time: 5.4070    pAUC: 0.4888 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 828540   FP: 26121   FN: 194   TP: 4     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 2.5364    Inference Time: 1.6449    pAUC: 0.5076 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 852632   FP: 2029    FN: 195   TP: 3     Params: {'n_bins': 8, 'n_random_cuts': 25}


# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 28 different hyperparameter combinations
 Training Time: 2.6630    Inference Time: 1.7938    pAUC: 0.5084 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 846379   FP: 8282    FN: 194   TP: 4     Params: {'n_bins': 6, 'n_random_cuts': 25}
 Training Time: 4.7723    Inference Time: 3.1468    pAUC: 0.5103 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 845826   FP: 8835    FN: 192   TP: 6     Params: {'n_bins': 6, 'n_random_cuts': 50}
 Training Time: 6.8845    Inference Time: 4.4906    pAUC: 0.5122 ± 0.0000   Recall: 0.0051 ± 0.0000   TN: 845990   FP: 8671    FN: 197   TP: 1     Params: {'n_bins': 6, 'n_random_cuts': 75}
 Training Time: 8.9872    Inference Time: 5.7952    pAUC: 0.5446 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 844877   FP: 9784    FN: 195   TP: 3     Params: {'n_bins': 6, 'n_random_cuts': 100}
 Training Time: 2.6522    Inference Time: 1.8126    pAUC: 0.5045 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 837411   FP: 17250   FN: 192   TP: 6     Params: {'n_bins': 8, 'n_random_cuts': 25}


# Training - COPOD

In [ ]:
# Local configuration
classifier_name = "copod"

# Define the classifier that will be used for training
classifier = COPOD(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 1 different hyperparameter combinations
 Training Time: 1.0519    Inference Time: 1.7257    pAUC: 0.6395 ± 0.0000   Recall: 0.0960 ± 0.0000   TN: 845647   FP: 9014    FN: 179   TP: 19    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.6395 ± 0.0000
Recall: 0.0960 ± 0.0000


# 2/6 - Features: activity, weekday

Testing 1 different hyperparameter combinations
 Training Time: 1.0060    Inference Time: 1.6838    pAUC: 0.5203 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 845824   FP: 8837    FN: 190   TP: 8     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5203 ± 0.0000
Recall: 0.0404 ± 0.0000


# 3/6 - Features: activity, hour, weekday

...

Testing 1 different hyperparameter combinations
 Training Time: 1.3091    Inference Time: 2.2379    pAUC: 0.7275 ± 0.0000   Recall: 0.3990 ± 0.0000   TN: 849059   FP: 5602    FN: 119   TP: 79    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7275 ± 0.0000
Recall: 0.3990 ± 0.0000


# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 1.2602    Inference Time: 2.1522    pAUC: 0.5017 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 847166   FP: 7495    FN: 198   TP: 0     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5017 ± 0.0000
Recall: 0.0000 ± 0.0000


# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 1.8583    Inference Time: 3.3304    pAUC: 0.7076 ± 0.0000   Recall: 0.3384 ± 0.0000   TN: 846069   FP: 8592    FN: 131   TP: 67    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7076 ± 0.0000
Recall: 0.3384 ± 0.0000


# Training - ECOD

In [ ]:
# Local configuration
classifier_name = "ecod"

# Define the classifier that will be used for training
classifier = ECOD(
    contamination=CONTAMINATION,
    n_jobs=1
)

# Define the hyperparameters grid that will be tested for best results
parameters = {}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 1 different hyperparameter combinations
 Training Time: 1.0537    Inference Time: 1.7274    pAUC: 0.7508 ± 0.0000   Recall: 0.4697 ± 0.0000   TN: 845179   FP: 9482    FN: 105   TP: 93    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7508 ± 0.0000
Recall: 0.4697 ± 0.0000


# 2/6 - Features: activity, weekday

Testing 1 different hyperparameter combinations
 Training Time: 1.0635    Inference Time: 1.7171    pAUC: 0.5070 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 847166   FP: 7495    FN: 198   TP: 0     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5070 ± 0.0000
Recall: 0.0000 ± 0.0000


# 3/6 - Features: activity, hour, weekday

...

Testing 1 different hyperparameter combinations
 Training Time: 1.3279    Inference Time: 2.2513    pAUC: 0.7463 ± 0.0000   Recall: 0.4091 ± 0.0000   TN: 848858   FP: 5803    FN: 117   TP: 81    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7463 ± 0.0000
Recall: 0.4091 ± 0.0000


# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 1.2510    Inference Time: 2.1516    pAUC: 0.5185 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 847166   FP: 7495    FN: 198   TP: 0     Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.5185 ± 0.0000
Recall: 0.0000 ± 0.0000


# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 1 different hyperparameter combinations
 Training Time: 1.8938    Inference Time: 3.3434    pAUC: 0.7028 ± 0.0000   Recall: 0.3030 ± 0.0000   TN: 846183   FP: 8478    FN: 138   TP: 60    Params: {}

Best hyperparameters:
Params: {}
pAUC:   0.7028 ± 0.0000
Recall: 0.3030 ± 0.0000


# Training - CBLOF

In [ ]:
# Local configuration
classifier_name = "cblof"

# Define the classifier that will be used for training
classifier = CBLOF(
    contamination=CONTAMINATION,
    n_jobs=N_JOBS
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_clusters": [2, 4],
    "alpha": [0.2, 0.4, 0.6, 0.8, 0.9],
    "beta": [2, 4, 8, 16],
    "use_weights": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 80 different hyperparameter combinations
 Training Time: 3.5336    Inference Time: 0.5129    pAUC: 0.5360 ± 0.0002   Recall: 0.1162 ± 0.0000   TN: 838451.0   FP: 16210.0   FN: 175.0   TP: 23.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 2.9131    Inference Time: 0.4369    pAUC: 0.5057 ± 0.0000   Recall: 0.0384 ± 0.0061   TN: 851508.0   FP: 3153.0   FN: 191.0   TP: 7.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 4.6587    Inference Time: 0.4262    pAUC: 0.6185 ± 0.0000   Recall: 0.1717 ± 0.0000   TN: 851896.0   FP: 2765.0   FN: 164.0   TP: 34.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 4.5363    Inference Time: 0.4603    pAUC: 0.7308 ± 0.0019   Recall: 0.1566 ± 0.0000   TN: 845312.0   FP: 9349.0   FN: 167.0   TP: 31.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 2.9531    Inference Time: 0.4427 

# 2/6 - Features: activity, weekday

Testing 80 different hyperparameter combinations
 Training Time: 3.3244    Inference Time: 0.4544    pAUC: 0.4966 ± 0.0000   Recall: 0.0101 ± 0.0000   TN: 847099.0   FP: 7562.0   FN: 196.0   TP: 2.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 3.2420    Inference Time: 0.4457    pAUC: 0.4934 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 846374.0   FP: 8287.0   FN: 190.0   TP: 8.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 4.5413    Inference Time: 0.4635    pAUC: 0.5022 ± 0.0006   Recall: 0.0172 ± 0.0040   TN: 828637.0   FP: 26024.0   FN: 193.0   TP: 5.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 4.4422    Inference Time: 0.4364    pAUC: 0.5444 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 846744.0   FP: 7917.0   FN: 198.0   TP: 0.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 3.2478    Inference Time: 0.4589    

# 3/6 - Features: activity, hour, weekday

...

Testing 80 different hyperparameter combinations
 Training Time: 3.4515    Inference Time: 0.4457    pAUC: 0.6031 ± 0.0000   Recall: 0.0758 ± 0.0000   TN: 853774.0   FP: 887.0   FN: 183.0   TP: 15.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 3.5753    Inference Time: 0.4521    pAUC: 0.6296 ± 0.0000   Recall: 0.0657 ± 0.0000   TN: 854000.0   FP: 661.0   FN: 185.0   TP: 13.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 5.4842    Inference Time: 0.4570    pAUC: 0.5232 ± 0.0199   Recall: 0.0586 ± 0.0272   TN: 848379.0   FP: 6282.0   FN: 182.0   TP: 16.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 5.0863    Inference Time: 0.4100    pAUC: 0.6210 ± 0.0045   Recall: 0.1051 ± 0.0198   TN: 840983.0   FP: 13678.0   FN: 182.0   TP: 16.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 3.3981    Inference Time: 0.4616  

# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 80 different hyperparameter combinations
 Training Time: 3.4154    Inference Time: 0.4315    pAUC: 0.4750 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 825554.0   FP: 29107.0   FN: 198.0   TP: 0.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 3.4285    Inference Time: 0.4370    pAUC: 0.4737 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 846872.0   FP: 7789.0   FN: 198.0   TP: 0.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 4.5228    Inference Time: 0.4417    pAUC: 0.4757 ± 0.0000   Recall: 0.0051 ± 0.0000   TN: 820092.0   FP: 34569.0   FN: 197.0   TP: 1.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 4.7081    Inference Time: 0.4496    pAUC: 0.4737 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 849609.0   FP: 5052.0   FN: 198.0   TP: 0.0   Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 3.4755    Inference Time: 0.4545   

# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 80 different hyperparameter combinations
 Training Time: 4.7855    Inference Time: 0.4842    pAUC: 0.5976 ± 0.0000   Recall: 0.0909 ± 0.0000   TN: 846295   FP: 8366    FN: 180   TP: 18    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': True}
 Training Time: 4.8974    Inference Time: 0.5031    pAUC: 0.6257 ± 0.0000   Recall: 0.0960 ± 0.0000   TN: 844990   FP: 9671    FN: 179   TP: 19    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 2, 'use_weights': False}
 Training Time: 6.8903    Inference Time: 0.4762    pAUC: 0.5965 ± 0.0000   Recall: 0.1010 ± 0.0000   TN: 845766   FP: 8895    FN: 178   TP: 20    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': True}
 Training Time: 6.9767    Inference Time: 0.4991    pAUC: 0.5976 ± 0.0002   Recall: 0.1162 ± 0.0000   TN: 843160   FP: 11501   FN: 175   TP: 23    Params: {'alpha': 0.2, 'beta': 2, 'n_clusters': 4, 'use_weights': False}
 Training Time: 5.2770    Inference Time: 0.5522    pAUC: 0.5976 ± 0.0000

# Training - PCA

In [ ]:
# Local configuration
classifier_name = "pca"

# Define the classifier that will be used for training
classifier = PCA(
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "n_components": [1, 2, 3, 4, 5, 6, 7],
    "whiten": [True, False],
    "svd_solver": ["full", "arpack", "randomized"],
    "weighted": [True, False],
    "standardization": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 168 different hyperparameter combinations
 Training Time: 0.6004    Inference Time: 0.4735    pAUC: 0.5543 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 844285   FP: 10376   FN: 190   TP: 8     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.5939    Inference Time: 0.4718    pAUC: 0.5543 ± 0.0000   Recall: 0.0404 ± 0.0000   TN: 844285   FP: 10376   FN: 190   TP: 8     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.5994    Inference Time: 0.4693    pAUC: 0.5609 ± 0.0000   Recall: 0.1313 ± 0.0000   TN: 848121   FP: 6540    FN: 172   TP: 26    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.6018    Inference Time: 0.4671    pAUC: 0.5609 ± 0.0000   Recall: 0.1313 ± 0.0000   TN: 848121   FP: 6540    FN: 172   TP: 26    Params: {'n_components': 1, 'stan

# 2/6 - Features: activity, weekday

Testing 168 different hyperparameter combinations
 Training Time: 0.5920    Inference Time: 0.4727    pAUC: 0.5244 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 845120   FP: 9541    FN: 194   TP: 4     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.6003    Inference Time: 0.4673    pAUC: 0.5244 ± 0.0000   Recall: 0.0202 ± 0.0000   TN: 845120   FP: 9541    FN: 194   TP: 4     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.6039    Inference Time: 0.4733    pAUC: 0.5244 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 847004   FP: 7657    FN: 195   TP: 3     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.5967    Inference Time: 0.4719    pAUC: 0.5244 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 847004   FP: 7657    FN: 195   TP: 3     Params: {'n_components': 1, 'stan

# 3/6 - Features: activity, hour, weekday

...

Testing 168 different hyperparameter combinations
 Training Time: 0.6611    Inference Time: 0.4929    pAUC: 0.5087 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 840780   FP: 13881   FN: 198   TP: 0     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.6545    Inference Time: 0.4878    pAUC: 0.5087 ± 0.0000   Recall: 0.0000 ± 0.0000   TN: 840780   FP: 13881   FN: 198   TP: 0     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.6633    Inference Time: 0.4899    pAUC: 0.6031 ± 0.0000   Recall: 0.2374 ± 0.0000   TN: 846759   FP: 7902    FN: 151   TP: 47    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.6625    Inference Time: 0.4920    pAUC: 0.6031 ± 0.0000   Recall: 0.2374 ± 0.0000   TN: 846759   FP: 7902    FN: 151   TP: 47    Params: {'n_components': 1, 'stan

# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 168 different hyperparameter combinations
 Training Time: 0.6675    Inference Time: 0.4979    pAUC: 0.4958 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 847004   FP: 7657    FN: 195   TP: 3     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.6654    Inference Time: 0.4892    pAUC: 0.4958 ± 0.0000   Recall: 0.0152 ± 0.0000   TN: 847004   FP: 7657    FN: 195   TP: 3     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.6696    Inference Time: 0.4908    pAUC: 0.4958 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 844975   FP: 9686    FN: 192   TP: 6     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.6728    Inference Time: 0.4930    pAUC: 0.4958 ± 0.0000   Recall: 0.0303 ± 0.0000   TN: 844975   FP: 9686    FN: 192   TP: 6     Params: {'n_components': 1, 'stan

# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 168 different hyperparameter combinations
 Training Time: 0.8848    Inference Time: 0.5256    pAUC: 0.5050 ± 0.0000   Recall: 0.0051 ± 0.0000   TN: 846000   FP: 8661    FN: 197   TP: 1     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': True}
 Training Time: 0.8976    Inference Time: 0.5251    pAUC: 0.5050 ± 0.0000   Recall: 0.0051 ± 0.0000   TN: 846000   FP: 8661    FN: 197   TP: 1     Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': True, 'whiten': False}
 Training Time: 0.8994    Inference Time: 0.5277    pAUC: 0.5704 ± 0.0000   Recall: 0.1465 ± 0.0000   TN: 846199   FP: 8462    FN: 169   TP: 29    Params: {'n_components': 1, 'standardization': True, 'svd_solver': 'full', 'weighted': False, 'whiten': True}
 Training Time: 0.8940    Inference Time: 0.5280    pAUC: 0.5704 ± 0.0000   Recall: 0.1465 ± 0.0000   TN: 846199   FP: 8462    FN: 169   TP: 29    Params: {'n_components': 1, 'stan

# Training - AE

In [ ]:
# Local configuration
classifier_name = "auto_encoder"

# Define the classifier that will be used for training
classifier = AutoEncoder(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[8, 4, 4, 8], [4, 2, 2, 4], [2, 1, 1, 2]],
    "hidden_activation": ["relu", "sigmoid", "tanh"]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 126.7097   Inference Time: 30.2098   pAUC: 0.6865 ± 0.0005   Recall: 0.1061 ± 0.0197   TN: 847407.0   FP: 7254.0   FN: 172.0   TP: 26.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 113.8671   Inference Time: 31.6119   pAUC: 0.6884 ± 0.0033   Recall: 0.0970 ± 0.0133   TN: 847612.0   FP: 7049.0   FN: 178.0   TP: 20.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 124.6573   Inference Time: 30.2384   pAUC: 0.6859 ± 0.0037   Recall: 0.1202 ± 0.0297   TN: 845334.0   FP: 9327.0   FN: 169.0   TP: 29.0   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Tra

# 2/6 - Features: activity, weekday

Testing 9 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 121.0327   Inference Time: 29.3839   pAUC: 0.4764 ± 0.0012   Recall: 0.0010 ± 0.0020   TN: 854661.0   FP: 0.0     FN: 198.0   TP: 0.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 110.8363   Inference Time: 29.4127   pAUC: 0.4806 ± 0.0083   Recall: 0.0000 ± 0.0000   TN: 846137.0   FP: 8524.0   FN: 198.0   TP: 0.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
Skipping {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}: The number of neurons should not exceed the number of features
 Training Time: 126.1090   Inference Time: 31.8072   pAUC: 0.5069 ± 0.0008   Recall: 0.0000 ± 0.0000   TN: 846137.0   FP: 8524.0   FN: 198.0   TP: 0.0   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [4, 2, 2, 4]}
 Trainin

# 3/6 - Features: activity, hour, weekday

...

Testing 9 different hyperparameter combinations
 Training Time: 176.5184   Inference Time: 30.6011   pAUC: 0.6033 ± 0.0282   Recall: 0.1879 ± 0.0274   TN: 843854.0   FP: 10807.0   FN: 150.0   TP: 48.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 157.2646   Inference Time: 32.4042   pAUC: 0.6542 ± 0.0010   Recall: 0.1646 ± 0.0069   TN: 842636.0   FP: 12025.0   FN: 164.0   TP: 34.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 128.5712   Inference Time: 35.4131   pAUC: 0.6515 ± 0.0004   Recall: 0.1657 ± 0.0074   TN: 842590.0   FP: 12071.0   FN: 164.0   TP: 34.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
 Training Time: 165.3038   Inference Time: 35.8880   pAUC: 0.6577 ± 0.0007   Recall: 0.1667 ± 0.0064   TN: 852675.0   FP: 1986.0   FN: 166.0   TP: 32.0   Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 151.5517   Inference Time: 35.1440

# 5/6 - Features: activity, weekday_sin, weekday_cos

Testing 9 different hyperparameter combinations
 Training Time: 163.9233   Inference Time: 32.2964   pAUC: 0.5299 ± 0.0098   Recall: 0.0000 ± 0.0000   TN: 846535.0   FP: 8126.0   FN: 198.0   TP: 0.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 159.7232   Inference Time: 33.4632   pAUC: 0.5255 ± 0.0115   Recall: 0.0000 ± 0.0000   TN: 846726.0   FP: 7935.0   FN: 198.0   TP: 0.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 127.9426   Inference Time: 32.7659   pAUC: 0.5384 ± 0.0040   Recall: 0.0000 ± 0.0000   TN: 846024.0   FP: 8637.0   FN: 198.0   TP: 0.0   Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
 Training Time: 190.9236   Inference Time: 37.2452   pAUC: 0.5325 ± 0.0138   Recall: 0.0000 ± 0.0000   TN: 845377   FP: 9284    FN: 198   TP: 0     Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 152.7387   Inference Time: 31.9384   pAUC: 0.5

# 6/6 - Features: activity, hour_sin, hour_cos, weekday_sin, weekday_cos

Testing 9 different hyperparameter combinations
 Training Time: 191.5450   Inference Time: 31.7827   pAUC: 0.6541 ± 0.0008   Recall: 0.1859 ± 0.0049   TN: 845529   FP: 9132    FN: 160   TP: 38    Params: {'hidden_activation': 'relu', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 161.8883   Inference Time: 34.8081   pAUC: 0.6467 ± 0.0015   Recall: 0.1939 ± 0.0025   TN: 846817   FP: 7844    FN: 159   TP: 39    Params: {'hidden_activation': 'relu', 'hidden_neurons': [4, 2, 2, 4]}
 Training Time: 160.1187   Inference Time: 32.4988   pAUC: 0.6429 ± 0.0010   Recall: 0.1919 ± 0.0000   TN: 845363   FP: 9298    FN: 160   TP: 38    Params: {'hidden_activation': 'relu', 'hidden_neurons': [2, 1, 1, 2]}
 Training Time: 211.4263   Inference Time: 35.5781   pAUC: 0.6551 ± 0.0006   Recall: 0.1919 ± 0.0000   TN: 847533   FP: 7128    FN: 160   TP: 38    Params: {'hidden_activation': 'sigmoid', 'hidden_neurons': [8, 4, 4, 8]}
 Training Time: 175.4747   Inference Time: 33.3541   pAUC: 0.6533 ± 0.0007   

# Training - Deep SVDD

In [ ]:
# Local configuration
classifier_name = "deep_svdd"

# Define the classifier that will be used for training
classifier = DeepSVDD(
    output_activation="sigmoid",
    optimizer=keras.optimizers.Adam(),
    epochs=100,
    batch_size=16384,
    validation_size=0.1,
    dropout_rate=0.2,
    l2_regularizer=0.1,
    preprocessing=False,
    verbose=0,
    callbacks=[
        keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=3, min_lr=1e-6),
        keras.callbacks.EarlyStopping(monitor="val_loss", patience=6)
    ],
    contamination=CONTAMINATION
)

# Define the hyperparameters grid that will be tested for best results
parameters = {
    "hidden_neurons": [[64, 32], [32, 16], [16, 8], [8, 4], [4, 2], [2, 1]],
    "hidden_activation": ["relu", "sigmoid", "tanh"],
    "use_ae": [True, False]
}

# Start the training
initiate_training_run(classifier_name, classifier, parameters)

# 1/6 - Features: activity, hour

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 157.6307   Inference Time: 28.6869   pAUC: 0.5737 ± 0.0437   Recall: 0.0576 ± 0.0472   TN: 846036   FP: 8625    FN: 170   TP: 28    Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_3083" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_3083/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_3083/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  •

# 2/6 - Features: activity, weekday

Testing 36 different hyperparameter combinations
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': True}: The number of neurons should not exceed the number of features
 Training Time: 148.3114   Inference Time: 29.7433   pAUC: 0.5081 ± 0.0311   Recall: 0.0121 ± 0.0104   TN: 847950   FP: 6711    FN: 195   TP: 3     Params: {'hidden_activation': 'relu', 'hidden_neurons': [64, 32], 'use_ae': False}
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': True}: The number of neurons should not exceed the number of features
Skipping {'hidden_activation': 'relu', 'hidden_neurons': [32, 16], 'use_ae': False}: Exception encountered when calling layer "tf.math.subtract_3566" (type TFOpLambda).

Dimensions must be equal, but are 16 and 32 for '{{node tf.math.subtract_3566/Sub}} = Sub[T=DT_FLOAT](Placeholder, tf.math.subtract_3566/Sub/y)' with input shapes: [?,16], [32].

Call arguments received:
  • x=tf.Tensor(shape=(None, 16), dtype=float32)
  •